# FINAL PROJECT

# player class for data

In [1]:

class Player():
    def __init__(self):
        """
        Initializes a Random Player that plays mancala
        """
        self.num_wins = 0
        self.num_loss = 0
        self.num_tie = 0
    
    def getAvgWins(self):
        """
        Calculates the average wins by dividing wins by number of games
        """
        return self.num_wins/(self.num_wins+self.num_loss+self.num_tie)
    
    def getAvgLosses(self):
        """
        Calculates the average loss by dividing losses by number of games
        """
        return self.num_loss/(self.num_wins+self.num_loss+self.num_tie)
    
    def getAvgTies(self):
        """
        Calculates the average ties by dividing ties by number of games
        """
        return self.num_tie/(self.num_wins+self.num_loss+self.num_tie)
    
    def display_stats(self):
        print("win percentage:", self.getAvgWins())
        print("loss percentage:", self.getAvgLosses())
        print("tie percentage:", self.getAvgTies())


# game class for playing

In [2]:
import random
import time
random.seed(time.process_time_ns())  # Seeds the random number generator for reproducibility

class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        Initializes the Mancala game with specified number of pits and stones per pit.
        """
        self.pits_per_player = pits_per_player  # Sets the number of pits per player
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2  # Number of players in the game
        # self.current_player = 1  # Initializes the current player to 1
        self.moves = []  # List to keep track of moves made
        self.p1_pits_index = [0, self.pits_per_player-1]  # Indexes for player 1's pits
        self.p1_mancala_index = self.pits_per_player  # Index for player 1's mancala
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]  # Indexes for player 2's pits
        self.p2_mancala_index = len(self.board)-1  # Index for player 2's mancala
        self.stones_per_pit = stones_per_pit
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0  # Sets player 1's mancala to 0
        self.board[self.p2_mancala_index] = 0  # Sets player 2's mancala to 0

    def display_board(self, board, current_player):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]  # Extracts player 1's pits
        player_1_mancala = board[self.p1_mancala_index]  # Extracts player 1's mancala
        player_2_pits = board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]  # Extracts player 2's pits
        player_2_mancala = board[self.p2_mancala_index]  # Extracts player 2's mancala

        print('P1               P2')  # Header for the board display
        print('     ____{}____     '.format(player_2_mancala))  # Displays player 2's mancala
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))  # Displays the last pit of player 1 and the first pit of player 2
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))  # Displays the pits of both players
            
        print('         {}         '.format(player_1_mancala))  # Displays player 1's mancala
        turn = 'P1' if current_player == 1 else 'P2'  # Determines whose turn it is
        print('Turn: ' + turn)  # Displays whose turn it is
        
    def valid_move(self, pit, board, current_player):
        """
        Checks if the pit chosen by the current_player is a valid move. TAKES IN 0-index for where pit is on board
        """
        if current_player==1: #For P1
            if pit<0 or pit>=self.p1_mancala_index or board[pit]<=0: #If not one of P1's pits or empty...
                return False #Invalid
        else: #For P2
            if pit<=self.p1_mancala_index or pit>=self.p2_mancala_index or board[pit]<=0: #If not one of P2's pits or empty...
                return False #Invalid
        return True  # If all checks pass, the move is valid
        
    def valid_moves(self, board, current_player):
        if current_player == 1: #For P1
            return [pit for pit in range(self.p1_mancala_index) if board[pit] > 0] #For every P1 pit other than their mancala...
        else: #For P2
            return [pit for pit in range(self.p1_mancala_index + 1, self.p2_mancala_index) if board[pit] > 0] #For every P2 pit other than their mancala...

    def random_move_generator(self, board, current_player):
        """
        Generates random valid moves with non-empty pits for the random player
        """
        
        pits = self.valid_moves(board, current_player)
        random_pit = random.randint(0, len(pits) - 1)
        return pits[random_pit] #Returns the random 0-index pit
    
    def _calculate_opposite_pit(self, current_pit):
        """Helper method to calculate the opposite pit index"""
        return self.p2_mancala_index - current_pit - 1

    def play(self, pit, board, current_player):
        """
        Simulates a single move made by a specific player using their selected pit. Input is a 0-index pit.
        """
        if not self.valid_move(pit, board, current_player):
            return -1  # Exits if the move is invalid
        
        stones = board[pit]  # Number of stones in the selected pit
        board[pit] = 0  # Empties the selected pit
        
        # Calculate the number of full cycles and remaining stones
        board_length = len(board)
        full_cycles = stones // (board_length - 1)  # -1 to account for skipping opponent's mancala
        remaining_stones = stones % (board_length - 1)
        
        # Distribute stones in bulk for full cycles
        if full_cycles > 0:
            for i in range(board_length):
                if (current_player == 1 and i == self.p2_mancala_index) or \
                   (current_player == 2 and i == self.p1_mancala_index):
                    continue
                board[i] += full_cycles
        
        # Distribute remaining stones
        current_pit = pit
        for _ in range(remaining_stones):
            current_pit = (current_pit + 1) % board_length
            if (current_player == 1 and current_pit == self.p2_mancala_index) or \
               (current_player == 2 and current_pit == self.p1_mancala_index):
                current_pit = (current_pit + 1) % board_length
            board[current_pit] += 1

        # Check for capture
        if board[current_pit] == 1:  # Only check if we ended with 1 stone
            if current_player == 1 and self.p1_pits_index[0] <= current_pit <= self.p1_pits_index[1]:
                opposite_pit = self._calculate_opposite_pit(current_pit)
                captured_stones = board[opposite_pit] + 1
                board[self.p1_mancala_index] += captured_stones
                board[current_pit] = 0
                board[opposite_pit] = 0
            elif current_player == 2 and self.p2_pits_index[0] <= current_pit <= self.p2_pits_index[1]:
                opposite_pit = self._calculate_opposite_pit(current_pit)
                captured_stones = board[opposite_pit] + 1
                board[self.p2_mancala_index] += captured_stones
                board[current_pit] = 0
                board[opposite_pit] = 0

        self.moves.append((current_player, pit + 1))
        current_player = 2 if current_player == 1 else 1
        
        return self.winning_eval(board, current_player) if self.winning_eval(board, current_player) != 0 else 0
        
    
    def winning_eval(self, board, current_player):
        """
        Evaluates if the game board has reached the winning state.
        """
        result = 0  # Initializes the result to 0
        if all(board[pit] <= 0 for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)):
            for pit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1):
                board[self.p2_mancala_index] += board[pit]  # Player 2 captures all stones in their pits
                board[pit] = 0  # Empties the pits
                result = 1  # Sets the result to 1 indicating a win for player 2
        
        if all(board[pit] <= 0 for pit in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)):
            for pit in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1):
                board[self.p1_mancala_index] += board[pit]  # Player 1 captures all stones in their pits
                board[pit] = 0  # Empties the pits
                result = 1  # Sets the result to 1 indicating a win for player 1
        
        if not result:
            return 0  # If no player has won, returns 0

        res1= board[self.p1_mancala_index]  # Player 1's mancala stones
        res2= board[self.p2_mancala_index]  # Player 2's mancala stones
        
        if res1 > res2: return 1  # Player 1 wins
        if res2 > res1: return 2  # Player 2 wins
        return 3  # It's a tie

# Alpha beta class for searching moves

In [3]:
class AB_OPTIMIZED:
    def __init__(self, man_obj) -> None:
        self.man_obj = man_obj
        
    def switch_player(self, player):
        """
        Switches the current player between 1 and 2.
        """
        return 2 if player == 1 else 1
        
    def utility(self, board: list, player=1):
        """
        Calculates the utility of an outcome for a player based on an integer representing a current state.
        """
        #TO-DO: See if this can/should be altered to work with difference in marbles rather than simply a winning state

        #Pros of just using terminal states: Easy, stops everything if a win is found
        #Pros of using difference in marbles: More complicated, may have to rework winning_eval or make a new method, could be much less expensive since a state of 32 to 4 may not be a winning state but is a good indicator of optimal play.
        if player==1:
            return board[self.man_obj.p1_mancala_index] - board[self.man_obj.p2_mancala_index]
        else:
            return board[self.man_obj.p2_mancala_index] - board[self.man_obj.p1_mancala_index]


    def max_value(self, board, alpha, beta, depth, player):
        """
        Calculates the maximum value the player can get out of this state.
        """
        
        # terminal = default_state.winning_eval()  # Calculates whether this state is a final state
        terminal = self.man_obj.winning_eval(board, player)
        
        if terminal > 0 or depth == 0:
            return self.utility(board, player), -1  # Calculate the utility of this state for the player whose current turn it is
        value = -1 * (self.man_obj.pits_per_player * self.man_obj.stones_per_pit * 2 + 1)
        pit_to_return = -1
        
        # pits = default_state.valid_moves()  # Figure out what actions are possible (0-index)
        pits = self.man_obj.valid_moves(board, player)
        for pit in pits:  # For every action...
            
            # state = copy.deepcopy(default_state)  # Clone the board state
            new_board = board.copy()
            
            # state.play(pit)  # Play move on this hypothetical board
            self.man_obj.play(pit, new_board, player)
            
            # pit_val, _ = self.min_value(state, alpha, beta, depth - 1, player)  # Determine value after making that move
            pit_val, _ = self.min_value(new_board, alpha, beta, depth - 1, self.switch_player(player))
            
            if pit_val > value:  # If the worst move your opponent can force you into in this state is better than the worst move they can force you into in another state...
                value = pit_val  # Select this value as our best value
                pit_to_return = pit  # Bookmark the pit or action chosen
            if value >= beta: return value, pit_to_return  # Prune if a value is found that is guaranteed to be chosen
            alpha = max(value, alpha)  # Calculate new alpha if needed
        return value, pit_to_return  # Return the value at this state


    def min_value(self, board, alpha, beta, depth, player):
        """
        Calculates the minimum value the player can get out of this state.
        """
        # terminal = default_state.winning_eval()  # Calculates whether this state is a final state
        terminal = self.man_obj.winning_eval(board, player)
        
        if terminal > 0 or depth == 0:
            return self.utility(board, player), -1  # Calculate the utility of this state for the player whose current turn it is
        
        value = self.man_obj.pits_per_player * self.man_obj.stones_per_pit * 2 + 1
        pit_to_return = -1
        
        # pits = default_state.valid_moves()  # Figure out what actions are possible (0-index)
        pits = self.man_obj.valid_moves(board, player)
        
        for pit in pits:  # For every action...
            # state = copy.deepcopy(default_state)  # Clone the board state
            new_board = board.copy()
            
            # state.play(pit)  # Play move on this hypothetical board
            self.man_obj.play(pit, new_board, player)
            
            
            # pit_val, _ = self.max_value(state, alpha, beta, depth - 1, player)  # Determine value after making that move
            pit_val, _ = self.max_value(new_board, alpha, beta, depth - 1, self.switch_player(player))  # Determine value after making that move
            
            
            if pit_val < value:  # If the best move the opponent can make in this state is worse than the best move they can make in another state...
                value = pit_val  # Select this value as our best value
                pit_to_return = pit  # Bookmark the pit or action chosen
            if value <= alpha: return value, pit_to_return  # Prune if a value is found that is guaranteed to be chosen
            beta = min(value, beta)  # Calculate new beta if needed
        return value, pit_to_return  # Return the value at this state


    def alphabeta_search(self, depth, player=1):
        """
        Performs AlphaBeta on the Mancala game.
        """
        
        beta = self.man_obj.pits_per_player * self.man_obj.stones_per_pit * 2 + 1
        alpha = -beta
        board = self.man_obj.board.copy()
        # state = copy.deepcopy(self)
        value, pit = self.max_value(board, alpha, beta, depth, player)
        return pit

# simulation class for testing and running

In [4]:
import time
from multiprocessing import Pool


def find_total_stones(game):
    total = 0
    for i in game.board:
        total += i
    return total


def play_rand_game(player1: Player, player2: Player, game: Mancala):
    playing = True
    curr_player = 1
    turns = 0
    
    while(playing):
        status = game.play(game.random_move_generator())
        # print(status, "stones: ", find_total_stones(game))
        # game.display_board()
        turns+=1
        if status < 0 :
            print("Failled")
            return turns
        
        if status > 0:
            print("Finished game")
            match status:
                case 1:
                    player1.num_wins += 1
                    player2.num_loss += 1
                    break
                case 2:
                    player1.num_loss += 1
                    player2.num_wins += 1
                    break
                case _:
                    player1.num_tie += 1
                    player2.num_tie += 1  
                    break   
        if curr_player == 1:
            curr_player = 2
        else:
            curr_player = 1
    return turns

def play_game_alphabeta(player1: Player, player2: Player, game: Mancala,depth: int=30):
    playing = True
    curr_player = 1
    turns = 0
    
    while(playing):
        status = game.play(game.alphabeta_search(depth))
        # print(status, "stones: ", find_total_stones(game))
        # game.display_board()
        turns+=1
        if status < 0 :
            print("Failled")
            return turns
        
        if status > 0:
            print("Finished game")
            match status:
                case 1:
                    player1.num_wins += 1
                    player2.num_loss += 1
                    break
                case 2:
                    player1.num_loss += 1
                    player2.num_wins += 1
                    break
                case _:
                    player1.num_tie += 1
                    player2.num_tie += 1  
                    break   
        if curr_player == 1:
            curr_player = 2
        else:
            curr_player = 1
    return turns

def play_game_alphabeta_rand(player1: Player, player2: Player, game:Mancala, depth: int = 10):
    playing = True
    curr_player = 1
    turns = 0
    
    
    
    while(playing):
        
        ab_obj = AB_OPTIMIZED(game)
        
        if curr_player==1:
            status = game.play(ab_obj.alphabeta_search(depth), game.board, 1)
        if curr_player==2:
            status = game.play(game.random_move_generator(game.board, 2),game.board, 2)
        # print(status, "stones: ", find_total_stones(game))
        # game.display_board()
        turns+=1
        if status < 0 :
            print("Failed")
            return turns
        
        if status > 0:
            print("Finished game")
            match status:
                case 1:
                    player1.num_wins += 1
                    player2.num_loss += 1
                    break
                case 2:
                    player1.num_loss += 1
                    player2.num_wins += 1
                    break
                case _:
                    player1.num_tie += 1
                    player2.num_tie += 1  
                    break   
        if curr_player == 1:
            curr_player = 2
        else:
            curr_player = 1
    return turns


def main():
    
    
    start_time = time.time()
    
    player1, player2 = Player(), Player()
    total_turns = 0
    
    
    for i in range(50):
        
        game = Mancala()
        total_turns += play_game_alphabeta_rand(player1, player2, game, 8)
        
        
    print("==============================player1==============================")
    player1.display_stats()
    print("==============================player2==============================")
    player2.display_stats()
    print("=============================game stats============================")
    print("average turns per game: {}".format(total_turns/(i+1)))
        # print(game.display_board())
        # print(i)
    # print(game.display_board())
    
    print("runtime: ", time.time() - start_time)


def play_game_and_update_stats(args):
        game, depth = args
        # Create new player instances for each process
        p1, p2 = Player(), Player()
        turns = play_game_alphabeta_rand(p1, p2, game, depth)
        return {
            'turns': turns,
            'p1_wins': p1.num_wins,
            'p1_losses': p1.num_loss,
            'p1_ties': p1.num_tie,
            'p2_wins': p2.num_wins,
            'p2_losses': p2.num_loss,
            'p2_ties': p2.num_tie
        }
        
def new_main():
    start_time = time.time()

    num_games = 5
    depth = 4
    
    # Create game instances and depth parameters
    games = [Mancala() for _ in range(num_games)]
    depths = [depth] * num_games
    args = list(zip(games, depths))

    # Create a pool and run the games
    with Pool() as p:
        results = p.map(play_game_and_update_stats, args)

    # Aggregate results
    total_turns = 0
    total_p1_wins = 0
    total_p1_losses = 0
    total_p1_ties = 0
    total_p2_wins = 0
    total_p2_losses = 0
    total_p2_ties = 0

    for i, result in enumerate(results):
        total_turns += result['turns']
        total_p1_wins += result['p1_wins']
        total_p1_losses += result['p1_losses']
        total_p1_ties += result['p1_ties']
        total_p2_wins += result['p2_wins']
        total_p2_losses += result['p2_losses']
        total_p2_ties += result['p2_ties']

        print(f"\nGame {i+1} Results:")
        print("==============================player1==============================")
        print(f"Wins: {total_p1_wins}, Losses: {total_p1_losses}, Ties: {total_p1_ties}")
        print("==============================player2==============================")
        print(f"Wins: {total_p2_wins}, Losses: {total_p2_losses}, Ties: {total_p2_ties}")
        print("=============================game stats============================")
        print(f"Average turns per game: {total_turns/(i+1)}")

    
    print("time: ", time.time() - start_time)


if __name__ == "__main__":
    main()
    # new_main()




Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
Finished game
==============================player1==============================
win percentage: 0.96
loss percentage: 0.04
tie percentage: 0.0
==============================player2==============================
win percentage: 0.04
loss percentage: 0.96
tie percentage: 0.0
=============================game stat

# some results using multithreading and different params
